In [8]:
from sqlalchemy import create_engine
from config import Config
from pandas import read_sql
from utils import BinaryEncoder
import datetime

In [2]:
eng = create_engine(Config.DATABASE_URI)
car_heat = read_sql(sql='select * from car_heat', con=eng)

encoder = BinaryEncoder()
cat_cols = ['a_rider', 'b_rider', 'c_rider', 'd_rider', 'stadium', 'round', 'year', 'time', 'name_team_home', 'name_team_away']
encoder.fit(data=car_heat, columns=cat_cols)

In [3]:
encoder.transform(data=car_heat)

In [4]:
encoder.save('test.txt')

TypeError: Object of type time is not JSON serializable

In [11]:
encoder.col_cats['time'].strftime("%H:%M")

AttributeError: 'list' object has no attribute 'strftime'

In [12]:
encoder.col_cats['time'] = [x.strftime("%H:%M") for x in encoder.col_cats['time']]

In [9]:
type(encoder.col_cats['time'][0]) == datetime.time

False